In [2]:
# First create the kh2lib object
from kh2lib.kh2lib import kh2lib
import os, json
lib = kh2lib()
if not os.path.isdir("workspace"):
    os.mkdir("workspace")

In [11]:
# Now we extract the xemnas BAR into a folder in our workspace
MDLX = "tt04.ard"
BAR = os.path.join(lib.gitpath, "KH2","ard",MDLX)
assert os.path.isfile(BAR)
lib.editengine.bar_extract(BAR, os.path.join(os.getcwd(), "workspace","tt04"))

In [24]:
lib.reset_git()

Git repository has been reset


In [6]:
MDLX = "tt04.ard"
BAR = os.path.join(lib.gitpath, "KH2","ard",MDLX)
lib.editengine.bar_build(os.path.join(os.getcwd(), "workspace", "tt04"), BAR)

In [ ]:
# Open the workspace in a window and copy the b_ex.ai to have an unmodified copy to diff against
# Then open the b_ex.ai in a hex editor and make the following changes to the bottom of the file
# Find the header definition for these two functions, and replace them with 00s
#  warp_building_front
#  rc_invitation_to_dark
# Alternatively, just edit a decompiled version of this AI file in Ghidra (using Govanify's KH2 plugin)

base_file = os.path.join("workspace", "battle", "enmp.list.base")
mod_file = os.path.join("workspace", "riku", "n_ex.ai")
assert os.path.isfile(base_file)
assert os.path.isfile(mod_file)

# go to the roxas fight, and create a memdump using emuhaste
#  it is recommended to make a savestate in the cutscene before the fight loads, for ease of testing
# Place the memdump in the workspace folder, using whatever name you choose
dumpname = "realriku.dump"
subdata = lib.parsingengine.load_memdump(os.path.join("workspace", dumpname))
# SUBDATA = lib.parsingengine.load_memdump(base_file) #IF THINGS ARENT WORKING LOOK HERE SILLY

starting_offset = int(lib.parsingengine.locate_file(base_file),16)
print(hex(starting_offset))
0/0
lib.cheatengine.pnach = []
# This code will diff the two versions of the ai file and create codes to make those same changes via RAM
codes = lib.parsingengine.generate_codes(str(starting_offset), base_file, mod_file)
for code in codes:
    print(code)
print(codes)
# The memdump skips the first section of memory addresses so the code offsets need to be fixed
codes = [code[1].replace("0","2") + code[1:] for code in codes]
codes
# codes = []#DEBUG

# Now we add the codes into the pnach portion of kh2lib, and generate a pnach file in your pcsx 2 directory
# Then just load your savestate, skip cutscene and start the fight
lib.cheatengine.apply_ram_code(codes)
lib.cheatengine.write_pnach(debug=True)

In [ ]:
import time, os
while 1:
    time.sleep(0.1)
    if os.path.isfile(mod_file):
        lib.cheatengine.pnach = []
        codes = lib.parsingengine.generate_codes(str(starting_offset), base_file, mod_file)
        codes = [code[1].replace("1","2") + code[1:] for code in codes]
        lib.cheatengine.apply_ram_code(codes)
        lib.cheatengine.write_pnach(debug=False)
        print(time.time())
        os.remove(mod_file)

In [ ]:
# Spawnscript reader
opcodes = {
    0x0: ("Spawn", "Load Spawn File"),
    0x1: ("MapOcclusion", "Enable/Disable meshes/collisions"),
    0x2: ("RandomSpawn", "Randomly spawn enemies"),
    0x3: ("CasualSpawn", "Casually spawn enemy group"),
    0x4: ("Capacity", "Set capacity of map (float)"),
    0x5: ("AllocEnemy", "Memory reserved for enemies"),
    0x6: ("Unk06", "Set memory area 0034ecd8"),
    0x7: ("Unk07", "Set the memory area 0034ecdc"),
    0x9: ("UncommonSpawn", "Somewhat like Spawn, used less"),
    0xa: ("Unk0a", "No Idea"),
    0xb: ("BarrierFlag", "Defines parts of the map that are blocked by a barrier"),
    0xc: ("AreaSettings", "Play a function related to evt"),
    0xe: ("Unk0e", "Unknown"),
    0xf: ("Party", "Which party members are allowed"),
    0x10: ("BGM", "Set Background Music"),
    0x11: ("MSGWall", "Sets message played when hitting invisible wall"),
    0x12: ("Unk12", "Unknown and unused"),
    0x13: ("Camera", "Camera mode for map"),
    0x14: ("StatusFlag3", "Sets 3rd bit of memory area 0034F240"),
    0x15: ("Mission", "Loads MSN file"),
    0x16: ("Bar", "Loads BAR file by specifying path"),
    0x17: ("StatusFlag5", "Sets 5th byte of memory area 0034F240 (Tornado related?)"),
    0x18: ("AllocEffect", "Allocates memory for the particle system?"),
    0x19: ("Progress", "Set story flag"),
    0x1a: ("Visibility On", "Something with 01c60548"),
    0x1b: ("Visibility Off", "Something with 01c60550"),
    0x1c: ("If", "Recursively calls spawnscript parser somehow?"),
    0x1d: ("Unk1d", "Unknown"),
    0x1e: ("BattleLevel", "Override battle level"),
    0x1f: ("Unk1F", "Unknown but similar to CasualSpawn")
}

In [12]:
SPAWNSCRIPT_PATH = os.path.join(os.getcwd(), "workspace", "tt04")
OUTDIR = os.path.join(os.getcwd(), "workspace", "tt04", "extracted_spawnscripts")
MAP_IN_PATH = os.path.join(SPAWNSCRIPT_PATH, "map.script")
MAP_OUT_PATH = os.path.join(OUTDIR, "map.txt")
EVT_IN_PATH = os.path.join(SPAWNSCRIPT_PATH, "evt.script")
EVT_OUT_PATH = os.path.join(OUTDIR, "evt.txt")
# BTL_IN_PATH = os.path.join(SPAWNSCRIPT_PATH, "btl.script")
# BTL_OUT_PATH = os.path.join(OUTDIR, "btl.txt")

In [13]:
lib.editengine.spawnscript_extract(MAP_IN_PATH, MAP_OUT_PATH)
lib.editengine.spawnscript_extract(EVT_IN_PATH, EVT_OUT_PATH)
# lib.editengine.spawnscript_extract(BTL_IN_PATH, BTL_OUT_PATH)

In [ ]:
SPAWNSCRIPT_PATH = os.path.join(os.getcwd(), "workspace", "tt01")
OUTDIR = os.path.join(os.getcwd(), "workspace", "tt01", "extracted_spawnscripts")
MAP_IN_PATH = os.path.join(SPAWNSCRIPT_PATH, "map.spawn")
MAP_OUT_PATH = os.path.join(OUTDIR, "map.txt")
EVT_IN_PATH = os.path.join(SPAWNSCRIPT_PATH, "evt.spawn")
EVT_OUT_PATH = os.path.join(OUTDIR, "evt.txt")
# BTL_IN_PATH = os.path.join(SPAWNSCRIPT_PATH, "btl.script")
# BTL_OUT_PATH = os.path.join(OUTDIR, "btl.txt")

In [ ]:
MDLX = "tt01.ard"
lib.reset_git()
BAR = os.path.join(lib.gitpath, "KH2","ard",MDLX)
assert os.path.isfile(BAR)
lib.editengine.bar_extract(BAR, os.path.join(os.getcwd(), "workspace","tt01"))
#lib.parsingengine.set_file_bytes(os.path.join(os.getcwd(), "workspace", "tt01", "100.event"))

In [ ]:
lib.editengine.bar_build(os.path.join(os.getcwd(), "workspace", "tt0","tt04.ard.json"), BAR)

In [22]:
lib.editengine.spawnscript_compile(MAP_OUT_PATH, MAP_IN_PATH)
lib.editengine.spawnscript_compile(EVT_OUT_PATH, EVT_IN_PATH)
lib.editengine.bar_build(os.path.join(os.getcwd(), "workspace", "tt04","tt04.ard.json"), BAR)
# lib.patch_game(fromgit=True)

In [ ]:
os.stat(MAP_OUT_PATH)

In [ ]:
os.stat(MAP_OUT_PATH)[8]

In [ ]:
# Using Mod Manager
import time, os
map_etime = 0
btl_etime = 0
evt_etime = 0
base_file_map = os.path.join("workspace", "tt01", "map.script")
mod_file_map = os.path.join("workspace", "tt01", "map.script.new")
# base_file_btl = os.path.join("workspace", "riku_ard", "btl.script")
# mod_file_btl = os.path.join("workspace", "riku_ard", "btl.script.new")
base_file_evt = os.path.join("workspace", "tt01", "evt.script")
mod_file_evt = os.path.join("workspace", "tt01", "evt.script.new")
num_failures = 0
last_exc = None
map_codes = []
evt_codes = []
btl_codes = []
while 1:
    if num_failures > 10:
        raise last_exc
    try:
        time.sleep(0.05)
        newCheats = False

        map_netime = os.stat(MAP_OUT_PATH)[8]
        evt_netime = os.stat(EVT_OUT_PATH)[8]
        if map_netime != map_etime:
            map_etime = map_netime
            newCheats = True
        if evt_netime != evt_etime:
            evt_etime = evt_netime
            newCheats = True
        if newCheats:
            lib.editengine.spawnscript_compile(MAP_OUT_PATH, MAP_IN_PATH)
            lib.editengine.spawnscript_compile(EVT_OUT_PATH, EVT_IN_PATH)
            lib.editengine.bar_build(os.path.join(os.getcwd(), "workspace", "tt01","tt01.ard.json"), BAR)
            print(time.time())
        num_failures = 0
        x=0
    except Exception as exc:
        last_exc = exc
        print(last_exc)
        num_failures += 1

In [ ]:
import time, os
map_etime = 0
btl_etime = 0
evt_etime = 0
base_file_map = os.path.join("workspace", "tt01", "map.script")
mod_file_map = os.path.join("workspace", "tt01", "map.script.new")
# base_file_btl = os.path.join("workspace", "riku_ard", "btl.script")
# mod_file_btl = os.path.join("workspace", "riku_ard", "btl.script.new")
base_file_evt = os.path.join("workspace", "tt01", "evt.script")
mod_file_evt = os.path.join("workspace", "tt01", "evt.script.new")
starting_offset_map = int(lib.parsingengine.locate_file(base_file_map),16)
starting_offset_btl = int(lib.parsingengine.locate_file(base_file_btl),16)
starting_offset_evt = int(lib.parsingengine.locate_file(base_file_evt),16)
num_failures = 0
last_exc = None
map_codes = []
evt_codes = []
btl_codes = []
while 1:
    if num_failures > 10:
        raise last_exc
    try:
        time.sleep(0.05)
        newCheats = False

        map_netime = os.stat(MAP_OUT_PATH)[8]
        btl_netime = os.stat(BTL_OUT_PATH)[8]
        evt_netime = os.stat(EVT_OUT_PATH)[8]
        if map_netime != map_etime:
            map_etime = map_netime
            newCheats = True
        if btl_netime != btl_etime:
            btl_etime = btl_netime
            newCheats = True
        if evt_netime != evt_etime:
            evt_etime = evt_netime
            newCheats = True
        if newCheats:
            lib.cheatengine.pnach = []
            lib.editengine.spawnscript_compile(MAP_OUT_PATH, MAP_IN_PATH+".new")
            map_codes = lib.parsingengine.generate_codes(str(starting_offset_map), base_file_map, mod_file_map)
            lib.editengine.spawnscript_compile(BTL_OUT_PATH, BTL_IN_PATH+".new")
            btl_codes = lib.parsingengine.generate_codes(str(starting_offset_btl), base_file_btl, mod_file_btl)
            lib.editengine.spawnscript_compile(EVT_OUT_PATH, EVT_IN_PATH+".new")
            evt_codes = lib.parsingengine.generate_codes(str(starting_offset_evt), base_file_evt, mod_file_evt)
            codes = [code[1].replace("1","2") + code[1:] for code in map_codes+btl_codes+evt_codes]
            lib.cheatengine.apply_ram_code(codes)
            lib.cheatengine.write_pnach(debug=False)
            print(time.time())
        num_failures = 0
    except Exception as exc:
        last_exc = exc
        print(last_exc)
        num_failures += 1

In [ ]:
# Extract every ARD
# Extract the Evt Spawnscript
# Record all the "SetJump", with the ard
# Put in a csv
# Once have all of them, randomize those lines, compile and build everything
from kh2lib.kh2lib import kh2lib
import os, json
lib = kh2lib()
if not os.path.isdir("workspace"):
    os.mkdir("workspace")
ardinfo = {}
alljumps = []
import os,shutil
ardparsedir = os.path.join(os.getcwd(), "workspace", "ardparse")
if os.path.exists(ardparsedir):
    shutil.rmtree(ardparsedir)
arddir = os.path.join(os.environ["USE_KH2_GITPATH"], "KH2", "ard")
for ard in os.listdir(arddir):
    fn = os.path.join(arddir, ard)

    out_pth = os.path.join(ardparsedir, os.path.basename(ard))
    lib.editengine.bar_extract(fn, out_pth)
    
    evtname = None
    for f in os.listdir(out_pth):
        if f.startswith("evt."):
            evtname = f
    ardinfo[ard] = {
        "fn": fn,
        "out_pth": out_pth
    }
    if evtname == None:
        print("ARD {} has no evt".format(ard))
    else:
        evt_pth = os.path.join(out_pth, evtname)
        lib.editengine.spawnscript_extract(evt_pth, evt_pth+".txt")
        jumps = []
        lines = open(evt_pth+".txt").read().split("\n")
        for line in lines:
            if "SetJump" in line:
                jumps.append(line)
                alljumps.append(line)
        ardinfo[ard]["evt_pth"] = evt_pth
        ardinfo[ard]["jumps"] = jumps
        ardinfo[ard]["lines"] = lines
        

In [ ]:
import random
alljumps = list(alljumps_old)
random.shuffle(alljumps)

for ard in ardinfo:
    if "jumps" in ardinfo[ard]:
        with open(ardinfo[ard]["evt_pth"]+".txt", "w") as f:
            for line in ardinfo[ard]["lines"]:
                if "SetJump" in line:
                    f.write("{}\n".format(alljumps.pop()))
                else:
                    f.write("{}\n".format(line))
        lib.editengine.spawnscript_compile(ardinfo[ard]["evt_pth"]+".txt", ardinfo[ard]["evt_pth"])
        lib.editengine.bar_build(os.path.join(ardinfo[ard]["out_pth"],ard+".json"), ardinfo[ard]["fn"])
with open(os.path.join(os.getcwd(), "workspace", "ardinfo.csv"), "w") as f:
    for ard in ardinfo:
        if "jumps" in ardinfo[ard]:
            for j in ardinfo[ard]["jumps"]:
                f.write("{},{}\n".format(ard, j))
alljumps

In [ ]:
### Testing BOss Rush Stuff
tt01_path = os.path.join(os.environ["USE_KH2_GITPATH"], "KH2", "tt01")
lib.editengine.spawnscript_compile(os.path.join(tt01_path,"extracted_spawnscripts","evt.txt"),os.path.join(tt01_path,"evt.script"))

In [1]:
## Cutscene Removal Mod
# Extract every ARD
# Extract the Evt Spawnscript
# Record all the "SetJump", with the ard
# Put in a csv
# Once have all of them, randomize those lines, compile and build everything
from kh2lib.kh2lib import kh2lib
import os, json
lib = kh2lib()
if not os.path.isdir("workspace"):
    os.mkdir("workspace")
ardinfo = {}
alljumps = []
import os,shutil
ardparsedir = os.path.join(os.getcwd(), "workspace", "ardparse")
if os.path.exists(ardparsedir):
    shutil.rmtree(ardparsedir)
arddir = os.path.join(os.environ["USE_KH2_GITPATH"], "KH2", "ard")
for ard in os.listdir(arddir):
    fn = os.path.join(arddir, ard)

    out_pth = os.path.join(ardparsedir, os.path.basename(ard))
    lib.editengine.bar_extract(fn, out_pth)
    
    evtname = None
    for f in os.listdir(out_pth):
        if f.startswith("evt."):
            evtname = f
    ardinfo[ard] = {
        "fn": fn,
        "out_pth": out_pth
    }
    if evtname == None:
        print("ARD {} has no evt".format(ard))
    else:
        evt_pth = os.path.join(out_pth, evtname)
        lib.editengine.spawnscript_extract(evt_pth, evt_pth+".txt")
        jumps = []
        lines = open(evt_pth+".txt").read().split("\n")
        for line in lines:
            if "SetJump" in line:
                jumps.append(line)
                alljumps.append(line)
        ardinfo[ard]["evt_pth"] = evt_pth
        ardinfo[ard]["jumps"] = jumps
        ardinfo[ard]["lines"] = lines
import random
alljumps = list(alljumps_old)
random.shuffle(alljumps)

ARD al01.ard has no evt
ARD al08.ard has no evt
ARD al12.ard has no evt
ARD di00.ard has no evt
ARD hb14.ard has no evt
ARD hb23.ard has no evt
ARD hb28.ard has no evt
ARD hb29.ard has no evt
ARD hb30.ard has no evt
ARD hb31.ard has no evt
ARD hb35.ard has no evt
ARD hb36.ard has no evt
ARD hb37.ard has no evt
ARD hb39.ard has no evt
ARD hb40.ard has no evt
ARD po10.ard has no evt
ARD tr06.ard has no evt
ARD tr11.ard has no evt


NameError: name 'alljumps_old' is not defined

In [5]:
for ard in ardinfo:
    if "jumps" in ardinfo[ard]:
        with open(ardinfo[ard]["evt_pth"]+".txt", "w") as f:
            for line in ardinfo[ard]["lines"]:
                if "SetEvent" in line:
                    f.write("	SetProgressFlag 0xFFFF\n")
                else:
                    f.write("{}\n".format(line))
        lib.editengine.spawnscript_compile(ardinfo[ard]["evt_pth"]+".txt", ardinfo[ard]["evt_pth"])
        lib.editengine.bar_build(os.path.join(ardinfo[ard]["out_pth"],ard+".json"), ardinfo[ard]["fn"])
with open(os.path.join(os.getcwd(), "workspace", "ardinfo.csv"), "w") as f:
    for ard in ardinfo:
        if "jumps" in ardinfo[ard]:
            for j in ardinfo[ard]["jumps"]:
                f.write("{},{}\n".format(ard, j))
alljumps

['\tSetJump Type 2 World AL Area 0 Entrance 0 LocalSet 117 FadeType 1',
 '\tSetJump Type 4 World AL Area 0 Entrance 0 LocalSet 0 FadeType -1',
 '\tSetJump Type 1 World AL Area 0 Entrance 53 LocalSet 0 FadeType 1',
 '\tSetJump Type 1 World AL Area 2 Entrance 0 LocalSet 0 FadeType 1',
 '\tSetJump Type 2 World AL Area 0 Entrance 0 LocalSet 57 FadeType 1',
 '\tSetJump Type 2 World AL Area 0 Entrance 0 LocalSet 118 FadeType 1',
 '\tSetJump Type 2 World AL Area 0 Entrance 0 LocalSet 115 FadeType 1',
 '\tSetJump Type 1 World AL Area 0 Entrance 51 LocalSet 0 FadeType 1',
 '\tSetJump Type 1 World AL Area 0 Entrance 52 LocalSet 0 FadeType 1',
 '\tSetJump Type 1 World AL Area 0 Entrance 54 LocalSet 0 FadeType 1',
 '\tSetJump Type 2 World AL Area 0 Entrance 0 LocalSet 113 FadeType 1',
 '\tSetJump Type 1 World AL Area 0 Entrance 50 LocalSet 0 FadeType 1',
 '\tSetJump Type 2 World AL Area 0 Entrance 0 LocalSet 127 FadeType 1',
 '\tSetJump Type 2 World AL Area 0 Entrance 0 LocalSet 114 FadeType 1',
 